In [ ]:
import pandas as pd 
import numpy as np
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import warnings 
from sklearn.metrics import mean_squared_error
import lofo
from sklearn.model_selection import StratifiedKFold, KFold
import seaborn as sns

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def stats_by_feat(train,test,feat_to_stat,feat_to_group,func):
    for df in (train,test):
        df[feat_to_stat+feat_to_group+func] = df[feat_to_group].map(df.groupby(feat_to_group)[feat_to_stat].aggregate(func))

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
train = train[~train['GT_NO2'].isna()]

In [ ]:
len(train)

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train['ID'].nunique()

In [ ]:
id = test['ID_Zindi']

In [ ]:
date = pd.to_datetime(train['Date'])

In [ ]:
feautures = ['LST','AAI','NO2_strat','NO2_total','NO2_trop','TropopausePressure']


In [ ]:

# for feature in feautures:
#     for stat in ['mean','std','min','max','var']:
#             stats_by_feat(train,test,feature,'ID',stat)

In [ ]:
for df in (train,test):
    df['Date'] = pd.to_datetime(df['Date'])
    
    df['day'] = df['Date'].dt.day
    df['day_sin'] = np.sin(df['Date'].dt.day)
    df['dayofyear']= df['Date'].dt.dayofyear
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['LON_sin'] = np.sin(df['LON'])
    df['Lat_sin'] = np.sin(df['LAT'])
    df['no_total_by_no_trop'] = df['NO2_total'] / df['NO2_trop']
    df.drop(columns=['Date','ID_Zindi','ID'],inplace=True)

In [ ]:

for feature in feautures:
    for group in ['day','dayofyear','month','year']:
        for stat in ['mean','std','min','max','var']:
            stats_by_feat(train,test,feature,group,stat)

In [ ]:
train

In [ ]:
test

In [ ]:
X = train.drop(columns='GT_NO2')
y = train['GT_NO2']

model = CatBoostRegressor(random_state=42)

cv = StratifiedKFold(n_splits=15,shuffle=True,random_state=42)

In [ ]:
train

In [ ]:
test

In [ ]:
import sklearn

In [ ]:
train

In [ ]:
train = train[['GT_NO2','LAT', 'LON', 'Precipitation', 'LST', 'AAI', 'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop', 'TropopausePressure', 'day', 'day_sin', 'dayofyear', 'month', 'year', 'LON_sin', 'no_total_by_no_trop', 'LSTdaymean', 'LSTdaystd', 'LSTdaymin', 'LSTdaymax', 'LSTdayvar', 'LSTdayofyearmean', 'LSTdayofyearstd', 'LSTdayofyearmin', 'LSTdayofyearmax', 'LSTdayofyearvar', 'LSTmonthmean', 'LSTmonthstd', 'LSTmonthmin', 'LSTmonthmax', 'LSTmonthvar', 'LSTyearmean', 'LSTyearstd', 'LSTyearmin', 'LSTyearmax', 'LSTyearvar', 'AAIdaymean', 'AAIdaystd', 'AAIdaymin', 'AAIdaymax', 'AAIdayvar', 'AAIdayofyearmean', 'AAIdayofyearstd', 'AAIdayofyearmin', 'AAIdayofyearmax', 'AAIdayofyearvar', 'AAImonthmean', 'AAImonthstd', 'AAImonthmin', 'AAImonthmax', 'AAImonthvar', 'AAIyearmean', 'AAIyearstd', 'AAIyearmin', 'AAIyearmax', 'AAIyearvar', 'NO2_stratdaymean', 'NO2_stratdaystd', 'NO2_stratdaymin', 'NO2_stratdaymax', 'NO2_stratdayvar', 'NO2_stratdayofyearmean', 'NO2_stratdayofyearstd', 'NO2_stratdayofyearmin', 'NO2_stratdayofyearmax', 'NO2_stratdayofyearvar', 'NO2_stratmonthmean', 'NO2_stratmonthstd', 'NO2_stratmonthmin', 'NO2_stratmonthmax', 'NO2_stratmonthvar', 'NO2_stratyearmean', 'NO2_stratyearstd', 'NO2_stratyearmin', 'NO2_stratyearmax', 'NO2_stratyearvar', 'NO2_totaldaymean', 'NO2_totaldaystd', 'NO2_totaldaymin', 'NO2_totaldaymax', 'NO2_totaldayvar', 'NO2_totaldayofyearmean', 'NO2_totaldayofyearstd', 'NO2_totaldayofyearmin', 'NO2_totaldayofyearmax', 'NO2_totaldayofyearvar', 'NO2_totalmonthmean', 'NO2_totalmonthstd', 'NO2_totalmonthmin', 'NO2_totalmonthmax', 'NO2_totalmonthvar', 'NO2_totalyearmean', 'NO2_totalyearstd', 'NO2_totalyearmin', 'NO2_totalyearmax', 'NO2_totalyearvar', 'NO2_tropdaymean', 'NO2_tropdaystd', 'NO2_tropdaymin', 'NO2_tropdaymax', 'NO2_tropdayvar', 'NO2_tropdayofyearmean', 'NO2_tropdayofyearstd', 'NO2_tropdayofyearmin', 'NO2_tropdayofyearmax', 'NO2_tropdayofyearvar', 'NO2_tropmonthmean', 'NO2_tropmonthstd', 'NO2_tropmonthmin', 'NO2_tropmonthmax', 'NO2_tropmonthvar', 'NO2_tropyearmean', 'NO2_tropyearstd', 'NO2_tropyearmin', 'NO2_tropyearmax', 'NO2_tropyearvar', 'TropopausePressuredaymean', 'TropopausePressuredaystd', 'TropopausePressuredaymin', 'TropopausePressuredaymax', 'TropopausePressuredayvar', 'TropopausePressuredayofyearmean', 'TropopausePressuredayofyearstd', 'TropopausePressuredayofyearmin', 'TropopausePressuredayofyearmax', 'TropopausePressuredayofyearvar', 'TropopausePressuremonthmean', 'TropopausePressuremonthstd', 'TropopausePressuremonthmin', 'TropopausePressuremonthmax', 'TropopausePressuremonthvar', 'TropopausePressureyearmean', 'TropopausePressureyearstd', 'TropopausePressureyearmin', 'TropopausePressureyearmax', 'TropopausePressureyearvar']]

In [ ]:
metrics = []
OOF = []
RV = []
def validate(trainset,testset,t):
    model.fit(trainset.drop(columns=t),trainset[t])
    pred = model.predict(testset.drop(columns=t))
    score = mean_squared_error(pred,testset[t],squared=False)
    OOF.extend(pred)
    RV.extend(testset[t])
    return score
for v_train,v_test in cv.split(train.drop(columns='GT_NO2'),date):
    train_v, test_v= train.iloc[v_train],train.iloc[v_test]
    rmse = validate(train_v,test_v,'GT_NO2')
    metrics.append(rmse)

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
len(RV)

In [ ]:
len(OOF)


In [ ]:
OOF = np.array(OOF).reshape(-1,1)

In [ ]:
OOF

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X=OOF,y=RV)

In [ ]:
np.array(metrics).mean()

In [ ]:
np.array(metrics).std()


In [ ]:
model.fit(train.drop(columns='GT_NO2'),train['GT_NO2'])

In [ ]:
# importance =pd.DataFrame({
#     'name':model.feature_name_,
#     'importance':model.feature_importances_
# })

In [ ]:
prediction = model.predict(test)

In [ ]:
linear_prediction = lr.predict(prediction.reshape(-1,1))

In [ ]:
prediction = linear_prediction

In [ ]:
sns.displot(prediction)

In [ ]:
submission = pd.DataFrame({"ID_Zindi":id,'GT_NO2':prediction})

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
model.feature_names_

In [ ]:
# import seaborn as sns 
# import matplotlib.pyplot as plt 

# plt.figure(figsize=(10,30))
# sns.barplot(x=importance['importance'],y=importance['name'])